# Quantum-inspired algorithms for numerical analysis

In this folder you will find the code that I used when writing the work [Quantum-inspired algorithms for multivariate analysis: from interpolation to partial differential equations](https://arxiv.org/abs/1909.06619). Part of this code is self-contained, implementing simple algorithms, such as the computation of entanglement measures or estimating interpolation errors; other parts depend on the [SeeMPS library](https://github.com/juanjosegarciaripoll/seemps). This notebook takes care of preparing the folder to host that and other libraries.

The list of files is as follows

- [00 Readme.ipynb](00%20Readme.ipynb). This file you are reading.
- [01 Exact samplings.ipynb](01%20Exact%20samplings.ipynb). Tools to encode discretized continuous functions into quantum states. Various 1D, 2D and 3D functions used throughout the work. Estimates of the entanglement in those encodings.
- [02 MPS discussion.ipynb](02%20MPS%20discussion.ipynb). Algorithms for encoding continuous functions using MPS states. Analysis of this representation for squeezed Gaussian states in 2D and 3D.
- [03 MPS Finite differences.ipynb](03%20MPS%20Finite%20differences.ipynb). As the name indicates, discussion of the encoding of differential operators and solution of Fokker-Planck equations using MPS.
- [04 MPS Fourier methods.ipynb](04%20MPS%20Fourier%20methods.ipynb). Discussion of the implementation of Quantum Fourier Transforms using MPS, its use for interpolation and for solving PDEs.
- [05 Plots.ipynb](05%20Plots.ipynb). Code to process the data computed by other notebooks into publication-ready plots.

This repository is organized with the following structure
- `./` The notebooks mentioned above and some helper scripts
- `seeq` Automatically downloaded library
- `data` Pickle files generated by the simulations of wavefunctions
- `data-mps` Pickle files generated by the simulations of MPS states
- `figures` Output directory for plots
- `jobs` Jobs that we send to our cluster when simulations are particularly heavy

This code download some required libraries

In [1]:
import os.path
if not os.path.exists('seemps'):
    !git clone http://github.com/juanjosegarciaripoll/seemps
!cd seemps && make.cmd all
for d in ['data','mps-data','figures']:
    if not os.path.exists(d):
        os.mkdir(d)

Making directories for mps\version.py
Writing file mps\version.py
Making directories for mps\__init__.py
Writing file mps\__init__.py
Making directories for mps\tools.py
Writing file mps\tools.py
Making directories for mps\test\tools.py
Writing file mps\test\tools.py
Making directories for mps\test\test_tools.py
Writing file mps\test\test_tools.py
Making directories for mps\state.py
Writing file mps\state.py
Making directories for mps\test\test_mps.py
Writing file mps\test\test_mps.py
Making directories for mps\test\test_sample_states.py
Writing file mps\test\test_sample_states.py
Making directories for mps\test\test_canonical.py
Writing file mps\test\test_canonical.py
Making directories for mps\mpo.py
Writing file mps\mpo.py
Making directories for mps\expectation.py
Writing file mps\expectation.py
Making directories for mps\test\test_expectation.py
Writing file mps\test\test_expectation.py
Making directories for mps\truncate.py
Writing file mps\truncate.py
Making directories for mps\t

The following is a Windows script used for the following tasks
1. `make all` extracts Python files from all notebooks. These files are used as tiny libraries in other parts, or they contain jobs that we submit to the cluster.
2. `make clean` eliminates files that can be recreated easily. This include scripts, libraries and data files.
3. `make cleanup` eliminates the output from all Jupyter notebooks. Only use it if you plan to run them all, which takes a lot of time.

In [2]:
%%writefile make.cmd
@echo off
if "%1" == "all" (
    python -c "import exportnb; import glob; exportnb.export_notebooks(glob.glob('*.ipynb'),verbose=True); quit()"
)
if "%1" == "clean" (
    rmdir /S /Q seeq
    del /Q make.cmd core_mps.py core.py mpi*.py job*.py
)
if "%1" == "cleanup" (
    for %%i in (*.ipynb); do jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace "%%i"
)

Overwriting make.cmd


In [3]:
!make.cmd all

Making directories for mpijobs.py
Writing file mpijobs.py
Making directories for core.py
Writing file core.py
Making directories for core_mps.py
Writing file core_mps.py
Making directories for mpi_mps_entropies2d.py
Writing file mpi_mps_entropies2d.py
Making directories for job-2d-m5.py
Writing file job-2d-m5.py
Making directories for job-2d-m7.py
Writing file job-2d-m7.py
Making directories for job-2d-m9.py
Writing file job-2d-m9.py
Making directories for job-2d-m11.py
Writing file job-2d-m11.py
Making directories for job-2d-m14.py
Writing file job-2d-m14.py
Making directories for job-2d-m16.py
Writing file job-2d-m16.py
Making directories for job-2d-m18.py
Writing file job-2d-m18.py
Making directories for mpi_mps_entropies3d.py
Writing file mpi_mps_entropies3d.py
Making directories for job-3d-m5.py
Writing file job-3d-m5.py
Making directories for job-3d-m7.py
Writing file job-3d-m7.py
Making directories for job-3d-m9.py
Writing file job-3d-m9.py
Making directories for job-3d-m11.py
W

## Tools

This code is used to manage our MPI jobs. It is a stupid class that spreads a number of simulations over various processors that have been coordinated using `mpirun`.

In [ ]:
# file: mpijobs.py
from mpi4py import MPI
import time
import pickle
import io
from contextlib import redirect_stdout

with io.StringIO() as buf, redirect_stdout(buf):
    print('redirected')
    output = buf.getvalue()

class Manager(object):

    def __init__(self, root=0, debug=False, root_computes=False):
        self.comm = MPI.COMM_WORLD
        self.inode = MPI.Get_processor_name()    # Node where this MPI process runs
        self.size = self.comm.Get_size()   # Size of communicator
        self.rank = self.comm.Get_rank()   # Ranks in communicator
        self.root = root
        self.isroot = self.rank == root
        self.root_computes = root_computes
        self.debug = debug
        if debug:
            name = 'master' if self.isroot else 'slave'
            print(f'Running {self.inode} [{name}], rank={self.rank} out of {self.size}')

    def partition(self, data):
        if self.root_computes:
            data = [data[i::self.size] for i in range(self.size)]
        else:
            data = [data[i::self.size-1] for i in range(self.size-1)]
            data = data[0:self.root] + [[]] + data[self.root:]
        return data

    def run(self, job_description, file=None):
        job_description = list(enumerate(job_description))
        if self.isroot:
            data = self.partition(job_description)
        else:
            data = None
        data = self.comm.scatter(data, root=self.root)
        if self.debug:
            print(f'** Node {self.rank} received {len(data)} items {[order for order,_ in data]}')
        data = [self.run_one(pair) for pair in data]
        if self.debug:
            print(f'** Node {self.rank} computed {len(data)} items')
        data = self.comm.gather(data, root=self.root)
        if self.isroot:
            data = sorted(sum(data, []), key=lambda x: x[0])
            if self.debug and self.isroot:
                print(f'** Root {self.rank} gathered {len(data)} items (expected {len(job_description)})')
            if file is not None:
                try:
                    clean = [value for order, time, value, text_output in data]
                    with open(file, 'ab') as f:
                        pickle.dump((clean, data), f)
                    if self.debug:
                        print(f'Master node {self.rank} saved data in {file}')
                except:
                    print(f'Unable to save data. Aborting')
            if self.debug:
                for order, time, _, text_output in data:
                    print(f'-----\nJob {order} output:')
                    print(text_output)
                    print(f'Ran in {time}s')

    def run_one(self, pair):
        order, job_item = pair
        t = time.process_time()
        output = ''
        with io.StringIO() as buf:
            with redirect_stdout(buf):
                try:
                    values = job_item[0](*job_item[1:])
                except Exception as e:
                    if debug:
                        print(f'Exception raised: "{e}"')
                    values = e
            text_output = buf.getvalue()
        return order, time.process_time() - t, values, text_output
